To run this project, `ollama` must be installed. This is a package that allows for LLM usage, easily. 

The `ollama` server can be run using `ollama serve`, for testing purposes. 

In [70]:
import ollama
# Check if models are installed or not.
try:
    print("Installed models on local ollama instance:")
    # print(ollama.list())
    INSTALLEDMODELS = []
    for modelName in ollama.list()["models"]:
        print(modelName["name"])
        INSTALLEDMODELS.append(modelName["name"])
except:
    raise Exception("ollama server is not online. \nUse ollama serve to run the ollama daemon.")

Installed models on local ollama instance:
phi3:mini


Papers generated use any notes included in the `NOTEFILE`. 

In [94]:
"""Check defaults and make sure there are no exceptions. """

MODELNAME = "phi3:mini"

if MODELNAME not in INSTALLEDMODELS:
    raise Exception(f"Model name {MODELNAME} not in installed models.")

NOTEFILE = "notes.txt"

try:
    with open(NOTEFILE, "r") as my_file:
        NOTES = "".join(my_file.readlines())
except FileNotFoundError:
    raise Exception(f"Could not find {NOTEFILE}.")


QUESTIONNUMBER = 5

if QUESTIONNUMBER > 5 or QUESTIONNUMBER < 2:
    raise Exception("Question amount not suitable.")

TOTALMARKS = 100

if TOTALMARKS // QUESTIONNUMBER < 1:
    raise Exception("Number of marks is too low.")

QUESTIONTYPE = "mcq"
""" This can differ between short, long and mcq."""

# try:
#     with open("exampleQuestions.txt", "r") as QUESTIONSFILE:
#         EXAMPLEQUESTIONS = "".join(QUESTIONSFILE.readlines())
# except OSError:
#     raise Exception("Question file does not exist.")

""" If true, then questions can be divided into a,b,c or i, ii, iii.
    Question subdivision support can be added later. 
"""
# if QUESTIONNUMBER > 10:
#     SUBDIVISIONS = True
# else:
#     SUBDIVISIONS = False



' If true, then questions can be divided into a,b,c or i, ii, iii.\n    Question subdivision support can be added later. \n'

In [110]:
class QuestionType:
    """This class houses the different types of questions, and the prompts, that can be created by the program for each paper.
    Parameters:
    name: str - the name of the question type.
    systemPrompt: str - the initial prompt that should be added to generate the questions.
    examplePrompt: str - any example questions that would aid in generation. 
    """

    def __init__(self, name: str, systemPrompt: str, examplePrompt: str = None) -> None:
        self.name = name
        self.systemPrompt = systemPrompt
        self.examplePrompt = examplePrompt


short = QuestionType(
    "short", "You must generate short questions that are clear to understand.")
long = QuestionType(
    "long", "You must generate long answer questions. Examples include essay questions.")
mcq = QuestionType(
    "mcq", "You must generate short multiple choice questions that can be answered using one sentence. These should be able to be answered efficiently. Do not return the answers or options.", )
ALLQUESTIONTYPES = {}

ALLQUESTIONTYPES[short.name] = short
ALLQUESTIONTYPES[long.name] = long
ALLQUESTIONTYPES[mcq.name] = mcq


if QUESTIONTYPE not in ALLQUESTIONTYPES.keys():
    raise Exception("Invalid question type.")

In [111]:
""" The system prompt should be designed to use few-shot learning in order to generate the most accurate questions for the source material. """


SYSTEMPROMPT = f"Your job is to generate {QUESTIONNUMBER} questions, in a list. Each question should be written as a string, and should be separated by one new line character \\n each. Questions should not be numbered. \nFor example, for 4 questions about famous authors would be generated as\n \"Which author beginning with the letter M contributed the most to American literary fiction?\"\\n\"When was this author born?\"\\n\"Which famous comet was associated with this author?\"\\n\"Did this author have a pen-name?\"\n"

if ALLQUESTIONTYPES[QUESTIONTYPE].examplePrompt != None:
  SYSTEMPROMPT += ALLQUESTIONTYPES[QUESTIONTYPE].systemPrompt + ALLQUESTIONTYPES[QUESTIONTYPE].examplePrompt
else:
  SYSTEMPROMPT += ALLQUESTIONTYPES[QUESTIONTYPE].systemPrompt

"""Adding notes specified by user."""
# if EXAMPLEQUESTIONS != "":
#     SYSTEMPROMPT += f"You must generate questions with a similar structure and form to the questions below: \n {EXAMPLEQUESTIONS}"
SYSTEMPROMPT += f"\nPlease generate the questions from these notes: \n {NOTES} \n"
print(SYSTEMPROMPT)

Your job is to generate 5 questions, in a list. Each question should be written as a string, and should be separated by one new line character \n each. Questions should not be numbered. 
For example, for 4 questions about famous authors would be generated as
 "Which author beginning with the letter M contributed the most to American literary fiction?"\n"When was this author born?"\n"Which famous comet was associated with this author?"\n"Did this author have a pen-name?"
You must generate short multiple choice questions that can be answered using one sentence. These should be able to be answered efficiently. Do not return the answers or options.
Please generate the questions from these notes: 
 A distributed system is a collection of autonomous computers linked by a network, with software to produce an integrated computing infrastructure. 
It is a set of discrete computers that perform a computation together, as if they were a single computing system, as a network of processes, which in

In [112]:
generation = ollama.generate(model=MODELNAME, prompt=SYSTEMPROMPT)
# print(generation["response"])
generated_questions = [question.replace(
    "\"", "") for question in generation["response"].split("\\n")]
# Filter out the blank spaces.
generated_questions = [question.strip() for question in generated_questions if question != ""]

# print(generated_questions)
for q in range(len(generated_questions)):
  print(f"Question [{q+1}] {generated_questions[q]}")


if len(generated_questions) != QUESTIONNUMBER:
  raise Exception(f"Problem with generated question amount: expected {QUESTIONNUMBER}, got {len(generated_questions)}")
 

# questionChoicesDict = {}
# for question in generated_questions:
#   CHOICEPROMPT = f"Given a question {question}, please generate short {MCQOPTIONS} options, in which {MCQOPTIONS - 1} is incorrect. Output all the options as comma separated strings. Each option should be comma separated. Use the notes {NOTES}. Do not output anything apart from the options.  "
#   generateChoices = ollama.generate(model=MODELNAME, prompt=CHOICEPROMPT)
#   questionChoicesDict[question] = generateChoices["response"]
#   print(generateChoices["response"])

['What is the primary purpose of a distributed system?', 'Which networking layer does interprocess communication rely on in distributed systems?', 'Name one protocol used for reliable end-to-end communication in distributed systems.', 'What role do middleware technologies like Java RMI play in distributed computing?', "Define the term 'model' as it relates to system abstractions and implementation details."]
Question [1] What is the primary purpose of a distributed system?
Question [2] Which networking layer does interprocess communication rely on in distributed systems?
Question [3] Name one protocol used for reliable end-to-end communication in distributed systems.
Question [4] What role do middleware technologies like Java RMI play in distributed computing?
Question [5] Define the term 'model' as it relates to system abstractions and implementation details.


In [121]:
if QUESTIONTYPE == 'mcq':
  """ Generate choices for each question."""
  MCQOPTIONS = 4
  QUESTIONDICT = {}
  for question in generated_questions:
    CHOICEPROMPT = f"Generate a short correct answer, that is less than 10 words, to the question {question}. This answer should be summarised and should not include any notes or formatting. \nFor example: for the question \"What is the capital of England?\" the output would be \"London\". \nFor example: for the question \"How many seas are there in the world\" the output would be \"7\". \nFor example: for the question \"What is an example of a JavaScript framework?\" the output would be \"React.js\""
    # CHOICEPROMPT = f"Please generate {MCQOPTIONS} different multiple choice answers for the question {question}. Each choice must be separated by a new line character \\n. \nFor example, for the question: What is the capital of Germany? \n The output should be: Cologne\\nFrankfurt\\nBerlin\\nHamburg. \nFor the question: How many moons does Earth have? The output could be: 1\\n2\\n4\\n3. Each choice should not be in a numbered list."
    # CHOICEPROMPT += f"Whenever possible, take the answer from the notes \n{NOTES}\n, but the answer should not be vague. "
    correctChoiceGeneration = ollama.generate(
        model=MODELNAME, prompt=CHOICEPROMPT)
    choiceList = [choice.replace(
        "\"", "") for choice in correctChoiceGeneration["response"].split("\\n") if choice != ""]
    QUESTIONDICT[str(question)] = choiceList
    print(question)
    print(choiceList)
    incorrectPrompt = f"Generate {MCQOPTIONS - 1} options, that are similar but not the same as {choiceList[0]}. \nFor example: for an option London, the output would be \"Edinburgh\\nGlasgow\\nAberdeen\\Leeds\". \nFor example: for an option \"7\" the output would be \"4\\n6\\n2\\n9\". \n"
    incorrectChoiceGeneration = ollama.generate(
        model=MODELNAME, prompt=incorrectPrompt)
    print(incorrectChoiceGeneration["response"].split("\\n"))
    
  
  

What is the primary purpose of a distributed system?
[' Enable resource sharing and scalability.']
[' Collaborative Integration and Elasticity Expansion\nDynamic Resource Allocation Networks\nInterconnected Infrastructure Adaptability']
Which networking layer does interprocess communication rely on in distributed systems?
[' Application layer typically handles interprocess communication in distributed systems.']
[' File System Management', 'Data Serialization and Deserialization', 'Distributed Resource Allocation Mechanisms"']
Name one protocol used for reliable end-to-end communication in distributed systems.
[' TCP (Transmission Control Protocol)']
[" 1. User Datagram Protocol (UDP): A simpler and connectionless protocol, UDP sacrifices reliability features of TCP for faster data transmission. It's widely used in real-time applications like VoIP or online gaming where occasional packet loss is acceptable.\n\n2. Stream Control Transmission Protocol (SCTP): SCTP is a transport layer pr

In [ ]:
""" Add generated questions to a tex file."""
try:
  with open("questions.tex", "w") as my_file:
      
      if QUESTIONTYPE == "mcq":   
        # my_file.write("\\begin{parts}\n")
        for question in generated_questions:  
          my_file.write(
              f"\\question[{TOTALMARKS // QUESTIONNUMBER}] {question} \n")
          my_file.write("\\begin{checkboxes} \n")
          # [my_file.write("\\choice {choices}") for choice in choices]
          my_file.write("\\end{checkboxes} \n")
          # my_file.write(f"\n")
        # my_file.write("\\end{parts}")
except OSError:
   raise Exception("questions.tex does not exist and cannot be written to.")